In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
from IPython.display import clear_output

In [ ]:
# !pip install torch torchvision
!pip install scikit-learn
!pip install openpyxl
!pip install lassonet
clear_output()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectFromModel

from lassonet import LassoNetClassifier



# Load data

In [ ]:
def load_data(train_path, test_path):
    # Load your data
    train_df = pd.read_csv(train_path, header=[0, 1]).fillna(0)
    test_df = pd.read_csv(test_path, header=[0, 1]).fillna(0)

    train_df.replace([np.inf, -np.inf], 0, inplace=True)
    test_df.replace([np.inf, -np.inf], 0, inplace=True)

    return train_df, test_df


def to_array(train_df, test_df, group=False):
    if not group:
        train_df.columns = train_df.columns.droplevel(level=0)
        test_df.columns = test_df.columns.droplevel(level=0)

    # Prepare the datasets
    X_train = train_df.drop(['label', 'filename'], axis=1).values
    y_train = train_df['label'].values
    X_val = test_df.drop(['label', 'filename'], axis=1).values
    y_val = test_df['label'].values

    return X_train, y_train, X_val, y_val

def to_array_2(train_df, test_df, group=False):
    if not group:
        train_df.columns = train_df.columns.droplevel(level=0)
        test_df.columns = test_df.columns.droplevel(level=0)

    # Prepare the datasets
    X_train = train_df.values
    X_val = test_df.values

    return X_train, X_val


def stylize(df, column_name):
    def color_gradient(val, min_val, max_val):
        # Normalize value
        normalized = (val - min_val) / ((max_val - min_val)+5e-12)
        # Calculate green intensity
        green_intensity = int(255 * normalized)
        return f'background-color: rgb({255 - green_intensity}, {255}, {255 - green_intensity})'

    #
    # Apply the color mapping
    styled_df = df.style.applymap(lambda x: color_gradient(x, df[column_name].min(), df[column_name].max()))
    return styled_df


In [ ]:
def svm(X_train, y_train, X_test, y_test):
    clf = SVC()
    clf.fit(X_train, y_train)
    all_predictions = clf.predict(X_test)
    f1 = f1_score(y_test, all_predictions)

    return f1

# Feature Selection

**1. All features**

In [ ]:
def replace_outliers_with_average(data):
    rows, cols = data.shape
    for col in range(cols):
        column_data = data[:, col]
        Q1 = np.quantile(column_data, 0.01)
        Q3 = np.quantile(column_data, 0.99)
        # IQR = Q3 - Q1
        lower_bound = Q1# - 1.5 * IQR
        upper_bound = Q3# + 1.5 * IQR

        # Identify outliers
        outliers_mask = (column_data < lower_bound) | (column_data > upper_bound)

        # Calculate the average of non-outliers
        non_outliers_avg = column_data[~outliers_mask].mean()
        # print(non_outliers_avg)

        # Replace outliers with the average of non-outliers
        data[outliers_mask, col] = non_outliers_avg

    return data

In [ ]:
train_path = '/content/gdrive/MyDrive/speech_analysis/train_with_groups.csv'
test_path = '/content/gdrive/MyDrive/speech_analysis/test_with_groups.csv'

train_df, test_df = load_data(train_path, test_path)
X_train_all_features, y_train, X_test_all_features, y_test = to_array(train_df.copy(), test_df.copy(), group=False)
group_names = set(train_df.columns.droplevel(level=1))
group_names.remove("Info")
feature_names = train_df.columns.droplevel(level=0).tolist()[:-2]




# X_train_all_features = replace_outliers_with_average(X_train_all_features.copy())
# X_test_all_features = replace_outliers_with_average(X_test_all_features.copy())
# Normalize features
scaler = MinMaxScaler()
X_train_all_features = scaler.fit_transform(X_train_all_features)
X_test_all_features = scaler.transform(X_test_all_features)

In [ ]:
train_df[""]

,SHIMMER_sma_max,SHIMMER_sma_min,SHIMMER_sma_span,SHIMMER_sma_maxPos,SHIMMER_sma_minPos,SHIMMER_sma_amean,SHIMMER_sma_linregc1,SHIMMER_sma_linregc2,SHIMMER_sma_linregerrA,SHIMMER_sma_linregerrQ,...,NHR_sma_de_quartile2,NHR_sma_de_quartile3,NHR_sma_de_iqr1_2,NHR_sma_de_iqr2_3,NHR_sma_de_iqr1_3,NHR_sma_de_percentile1,NHR_sma_de_percentile99,NHR_sma_de_pctlrange0_1,NHR_sma_de_upleveltime75,NHR_sma_de_upleveltime90
0,8.743803,0.000000,8.743803,411.0,194.0,0.599705,-0.000044,0.656096,1131.962111,880.588945,...,0.000000,0.018514,0.018359,0.018514,0.036873,-0.973510,0.863272,1.836782,24.990177,0.039293
1,1.968703,0.000000,1.968703,639.0,146.0,0.517245,-0.000059,0.543971,414.744676,288.128863,...,0.000000,0.015795,0.015340,0.015795,0.031135,-0.653519,0.808696,1.462215,25.000000,0.440529
2,2.011201,0.000000,2.011201,2072.0,156.0,0.545874,0.000059,0.473379,1082.188261,665.220134,...,0.000000,0.014758,0.014018,0.014758,0.028777,-0.658235,0.623070,1.281306,25.010134,0.040535
3,1.978170,0.000000,1.978170,521.0,10.0,0.456132,-0.000250,0.586490,407.974955,276.908301,...,0.000000,0.013994,0.014366,0.013994,0.028359,-2.050321,2.009058,4.059379,25.023969,0.095877
4,1.986762,0.000000,1.986762,940.0,192.0,0.547376,-0.000104,0.696484,1187.777441,740.534974,...,0.000000,0.018788,0.018378,0.018788,0.037167,-0.709038,0.650092,1.359131,24.991310,0.034758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,1.997364,0.014544,1.982819,3098.0,3260.0,0.647442,0.000040,0.557274,2366.082047,1659.918249,...,0.000000,0.019833,0.020506,0.019833,0.040339,-0.508568,0.549738,1.058305,25.010999,0.021997
162,1.978887,0.017957,1.960931,162.0,690.0,0.749451,0.000047,0.654048,1958.638222,1205.551630,...,0.000028,0.041015,0.040777,0.040987,0.081764,-2.884966,3.284399,6.169364,25.000000,0.049407
163,1.996077,0.009151,1.986925,2938.0,351.0,0.733755,0.000012,0.715771,1828.138652,1407.556364,...,0.000040,0.016267,0.015198,0.016227,0.031425,-0.571281,0.530441,1.101722,24.991875,0.097498
164,1.994377,0.015507,1.978870,1826.0,3116.0,0.553951,0.000071,0.440651,1194.610026,709.056117,...,0.000022,0.024090,0.023786,0.024068,0.047854,-1.282023,1.296912,2.578935,25.015596,0.031192


In [ ]:
feature_importances_df = pd.DataFrame(columns=train_df.columns[:-2])

In [ ]:
X_train_all_features.shape

(166, 6859)

In [ ]:
X_test_all_features.shape

(71, 6859)

**2. PCA**

In [ ]:
# 129 150 166
pca = PCA(n_components=166)
X_train_pca = pca.fit_transform(X_train_all_features)
X_test_pca = pca.transform(X_test_all_features)

**3. JMIM**

In [ ]:
train_path = '/content/gdrive/MyDrive/speech_analysis/JMIM_train_2.csv'
test_path = '/content/gdrive/MyDrive/speech_analysis/JMIM_test_2.csv'

X_train_JMIM, y_train_JMIM, X_test_JMIM, y_test_JMIM = load_data(train_path, test_path)

scaler = MinMaxScaler()
X_train_JMIM = scaler.fit_transform(X_train_JMIM)
X_test_JMIM = scaler.transform(X_test_JMIM)

In [ ]:
JMIM_feature_importance_df = pd.read_csv("/content/gdrive/MyDrive/speech_analysis/JMIM_feature_importances.csv", header=[0, 1], index_col=0)

**4. Lasso (L1 Regulizer)**

In [ ]:
def lasso(X_train, X_test, y_train, ratio=1):
    lsvc = LinearSVC(C=ratio, penalty="l1", dual=False).fit(X_train, y_train)

    model = SelectFromModel(lsvc, prefit=True)
    X_train_lasso = model.transform(X_train)
    X_test_lasso = model.transform(X_test)

    return X_train_lasso, X_test_lasso, np.abs(lsvc.coef_)

In [ ]:
for group in group_names:
    X_train, X_test = to_array_2(train_df[group].copy(), test_df[group].copy(), group=True)

    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train_lasso, X_test_lasso, feature_importances = lasso(X_train, X_test, y_train)

    selected_columns = [(group, feature_name) for feature_name in train_df[group].columns]
    feature_importances_df.loc["LASSO_GP_Based", selected_columns] = feature_importances.reshape(-1)

In [ ]:
X_train_lasso, X_test_lasso, feature_importances = lasso(X_train_all_features, X_test_all_features, y_train)
feature_importances_df.loc["LASSO", :] = feature_importances

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
feature_importances_df

Pausing behavior                                      \
               count_pause_segments hesitation_rate num_words_to_pauses   
LASSO_GP_Based             0.613157        1.483495            0.472191   
LASSO                           0.0        0.255254                 0.0   

                                                                  \
               pasue_speech_ratio pause_length pause_lengths_avg   
LASSO_GP_Based                0.0     1.246019               0.0   
LASSO                         0.0          0.0               0.0   

                                                                              \
               pause_speech_duration_ratio pause_to_syllable pause_to_tokens   
LASSO_GP_Based                    0.242809          0.952262             0.0   
LASSO                                  0.0               0.0             0.0   

                                        ...                   Complexity  \
               pause_totallength_ratio  ... AMP_ENTROPY_sma_de_quartile2   
LASSO_GP_Based                 1.60724  ...                      1.55025   
LASSO                              0.0  ...                          0.0   

                                                                       \
               AMP_ENTROPY_sma_de_quartile3 AMP_ENTROPY_sma_de_iqr1_2   
LASSO_GP_Based                          0.0                       0.0   
LASSO                                   0.0                       0.0   

                                                                    \
               AMP_ENTROPY_sma_de_iqr2_3 AMP_ENTROPY_sma_de_iqr1_3   
LASSO_GP_Based                       0.0                       0.0   
LASSO                                0.0                       0.0   

                                                                               \
               AMP_ENTROPY_sma_de_percentile1 AMP_ENTROPY_sma_de_percentile99   
LASSO_GP_Based                            0.0                             0.0   
LASSO                                     0.0                             0.0   

                                                \
               AMP_ENTROPY_sma_de_pctlrange0_1   
LASSO_GP_Based                             0.0   
LASSO                                      0.0   

                                                 \
               AMP_ENTROPY_sma_de_upleveltime75   
LASSO_GP_Based                              0.0   
LASSO                                       0.0   

                                                 
               AMP_ENTROPY_sma_de_upleveltime90  
LASSO_GP_Based                         0.367206  
LASSO                                       0.0  

[2 rows x 6859 columns]

**5. Lassonet**

In [ ]:
def lasso_net(X_train, X_test, y_train):
    model = LassoNetClassifier(M=10, hidden_dims=(50,), path_multiplier=1.1, verbose=False)
    path = model.path(X_train, y_train, return_state_dicts=True)

    return model.feature_importances_.numpy()

In [ ]:
for group in group_names:
    X_train, X_test = to_array_2(train_df[group].copy(), test_df[group].copy(), group=True)

    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    feature_importances = lasso_net(X_train, X_test, y_train)

    selected_columns = [(group, feature_name) for feature_name in train_df[group].columns]
    feature_importances_df.loc["LASSONET_GP_Based", selected_columns] = feature_importances.reshape(-1)

In [ ]:
feature_importances = lasso_net(X_train_all_features, X_test_all_features, y_train)
feature_importances_df.loc["LASSONET", :] = feature_importances

In [ ]:
feature_importances_df

Pausing behavior                                      \
                  count_pause_segments hesitation_rate num_words_to_pauses   
LASSO_GP_Based                     0.0        0.923188                 0.0   
LASSO                              0.0        0.621924                 0.0   
LASSONET_GP_Based           134.823212      179.449692          148.305527   
LASSONET                     11.022336       67.411606            5.656201   

                                                                     \
                  pasue_speech_ratio pause_length pause_lengths_avg   
LASSO_GP_Based                   0.0     0.191511               0.0   
LASSO                            0.0     0.041044               0.0   
LASSONET_GP_Based         122.566551   179.449692        179.449692   
LASSONET                    5.656201    50.647335          6.844003   

                                                                 \
                  pause_speech_duration_ratio pause_to_syllable   
LASSO_GP_Based                            0.0               0.0   
LASSO                                     0.0               0.0   
LASSONET_GP_Based                  134.823212        163.136078   
LASSONET                             6.221821          6.221821   

                                                           ...  \
                  pause_to_tokens pause_totallength_ratio  ...   
LASSO_GP_Based                0.0                     0.0  ...   
LASSO                         0.0                     0.0  ...   
LASSONET_GP_Based      163.136078              179.449692  ...   
LASSONET                 5.142001                8.281244  ...   

                                    Complexity                               \
                  AMP_ENTROPY_sma_de_quartile2 AMP_ENTROPY_sma_de_quartile3   
LASSO_GP_Based                             0.0                          0.0   
LASSO                                      0.0                          0.0   
LASSONET_GP_Based                    67.411606                    34.592812   
LASSONET                              6.221821                     5.656201   

                                                                       \
                  AMP_ENTROPY_sma_de_iqr1_2 AMP_ENTROPY_sma_de_iqr2_3   
LASSO_GP_Based                          0.0                       0.0   
LASSO                                   0.0                       0.0   
LASSONET_GP_Based                 46.043034                   28.5891   
LASSONET                           7.528404                  5.656201   

                                                                            \
                  AMP_ENTROPY_sma_de_iqr1_3 AMP_ENTROPY_sma_de_percentile1   
LASSO_GP_Based                          0.0                            0.0   
LASSO                                   0.0                            0.0   
LASSONET_GP_Based                 31.448011                      38.052094   
LASSONET                           8.281244                       4.674546   

                                                   \
                  AMP_ENTROPY_sma_de_percentile99   
LASSO_GP_Based                                0.0   
LASSO                                         0.0   
LASSONET_GP_Based                       31.448011   
LASSONET                                 5.656201   

                                                   \
                  AMP_ENTROPY_sma_de_pctlrange0_1   
LASSO_GP_Based                                0.0   
LASSO                                         0.0   
LASSONET_GP_Based                       31.448011   
LASSONET                                 4.674546   

                                                    \
                  AMP_ENTROPY_sma_de_upleveltime75   
LASSO_GP_Based                                 0.0   
LASSO                                          0.0   
LASSONET_GP_Based                        31.448011   
LASSONET                           

In [ ]:
all_feature_importances = pd.concat([feature_importances_df, JMIM_feature_importance_df])

**Compare Feature Importances**

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/speech_analysis/final_feature_importance.csv", header=[0, 1], index_col=0)
df = df.T

In [ ]:
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df = normalized_df.fillna(0)

In [ ]:
save_path = "/content/gdrive/MyDrive/Elyas_Sahab_Maryam_speech_analysis/Codes/JMIM-EmoBase/Feature Importances"
for group in group_names:
    normalized_df.loc[group].style.background_gradient().to_excel(f'{save_path}/{group}.xlsx', engine='openpyxl')

In [ ]:
np.sum((normalized_df["LASSO"] != 0.0).to_numpy())

95

**3. JMIM**

In [ ]:
svm(X_train_JMIM, y_train_JMIM, X_test_JMIM, y_test_JMIM)

0.6969696969696969

In [ ]:
s = []
for _ in range(5):
    s.append(train(X_train_JMIM, y_train_JMIM, X_test_JMIM, y_test_JMIM, 530))
print(f"{sum(s) / 5} \t {s}")

0.707556205351405 	 [0.7164179104477612, 0.6969696969696969, 0.7142857142857143, 0.7042253521126761, 0.7058823529411764]


**4. Lasso (L1 Regulizer)**

In [ ]:
res = []
for i in range(1, 100):
    X_train_lasso, X_test_lasso = lasso(i)
    res.append(svm(X_train_lasso, y_train, X_test_lasso, y_test))
print(max(res))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7536231884057971


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
np.argmax(res)

71

In [ ]:
X_train_lasso, X_test_lasso = lasso(72)
f1s = []
for _ in range(5):
    f1s.append(train(X_train_lasso, y_train, X_test_lasso, y_test, 530))

print(f"{sum(f1s) / 5} \t {f1s}")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7508704694003695 	 [0.7671232876712328, 0.7605633802816902, 0.7462686567164178, 0.7384615384615385, 0.7419354838709677]


**5. Lassonet**

In [ ]:
f1s = []
for save in path:
    model.load(save.state_dict)
    y_pred = model.predict(X_test_all_features)
    n_selected.append(save.selected.sum())
    f1s.append(f1_score(y_test, y_pred))
    lambda_.append(save.lambda_)

print(max(f1s))

0.7435897435897435
